In [1]:
# Remember you need to use SWAN with NXCALS support....
from datetime import datetime
from datetime import timedelta
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector as msql
import pytimber
import os
ldb = pytimber.LoggingDB(source="nxcals")
#this is the new pytimber access, not working yet
#ldb = pytimber.LoggingDB(spark_session = spark)
#ldb = pytimber.LoggingDB(spark_session = spark, sparkconf="large",
#    sparkprops={
#        "spark.driver.cores", "4",
#        "spark.executor.memory", "10g",
#        "spark.sql.parquet.columnarReaderBatchSize", "32",
#        "spark.driver.maxResultSize", "1t",
#        "spark.task.maxDirectResultSize", "1t"
#})

kinit: type the password for `regenfus`:········
Password for regenfus@CERN.CH: 


In [9]:
# Send last extraction settings to mysql
# routine has to be stopped by the run manager from mysql
# only extraction time to GBAR is taken into account 
# some data could be stale 
BeamInjection = 'LNR.APULB.0227:Acquisition:totalIntensitySingle' # Inj into AD
BeamExtrALPH  = 'LNE.APULB.0030:Acquisition:totalIntensitySingle' # Extr into 00 beam line (alpha,aegis,base ...
BeamExtrGBAR  = 'LNE.APULB.5030:Acquisition:totalIntensitySingle' # Extr into 50 beam line (gbar, puma ...
CycleName     = 'ELENA.LSA:CYCLE'

# open connection to gbar msql DB
gbdb = msql.connect(host='pcnometh28.dyndns.cern.ch',database='gbar',user='gbar',password='To2023moko') 
if gbdb.is_connected():
    print('Connected to Gbar mysql database')
    cursor = gbdb.cursor()  #cursor = gbsc.cursor(buffered=True)

# make sure script is starting    
cursor.execute("UPDATE man SET goElena =1")
gbdb.commit()

d    = timedelta(minutes = 3)   # to make sure that we have at least 2 extractions
ltrg = 0.0
go   = True
while go:                         #for i in range(10):  # 240
    now  = datetime.now()  
    dat  = ldb.get([BeamExtrALPH,BeamExtrGBAR,BeamInjection],now-d,now+d)
    kys  = list(dat)
    cyc  = ldb.get([CycleName],now-d,now+d)
    # print(cyc[CycleName][1][-1])
    # dict_keys(['LNE.APULB.0030:Acquisition:totalIntensitySingle',
    #            'LNE.APULB.5030:Acquisition:totalIntensitySingle',
    #            'LNR.APULB.0227:Acquisition:totalIntensitySingle'])    
    ntrg = dat[kys[1]][0][-1]                  # time of extr. to GBAR
    cursor.execute("SELECT goElena FROM man")  # check if we want to stop
    go   = bool(cursor.fetchone()[0])          # bool(1) = True   int(bool(1)) = 1
    gbdb.commit()
    #print(go)
    if ntrg > ltrg and go:
        dt = datetime.fromtimestamp(round(ntrg,0))
        print("we have a new trigger",dt,dat[kys[1]][0][-1],dat[kys[1]][1][-1],cyc[CycleName][1][-1]) # 2nd key last elem: tme_stmp, I
        ltrg = ntrg
        cursor.execute("UPDATE elena SET extr03tme =" + str(dat[kys[0]][0][-1]))
        cursor.execute("UPDATE elena SET extr03sig =" + str(dat[kys[0]][1][-1]))  
        cursor.execute("UPDATE elena SET extr53tme =" + str(dat[kys[1]][0][-1]))
        cursor.execute("UPDATE elena SET extr53sig =" + str(dat[kys[1]][1][-1]))    
        cursor.execute("UPDATE elena SET inj227tme =" + str(dat[kys[2]][0][-1]))    
        cursor.execute("UPDATE elena SET inj227sig =" + str(dat[kys[2]][1][-1]))    
        gbdb.commit()       
    time.sleep(2)            
    
if gbdb is not None and gbdb.is_connected():
    gbdb.close()
    print('Connection to mysql closed.')        
print("finished")

Connected to Gbar mysql database
we have a new trigger 2023-10-31 18:23:17 1698772997.333 6.81443977355957 NewOptics-Hminus_2023
we have a new trigger 2023-10-31 18:23:34 1698773014.133 10.347817420959473 NewOptics-Hminus_2023
we have a new trigger 2023-10-31 18:23:50 1698773029.733 6.798115253448486 LongInjection_Hminus_2023
we have a new trigger 2023-10-31 18:24:05 1698773045.333 11.46136474609375 NewOptics-Hminus_2023
we have a new trigger 2023-10-31 18:24:22 1698773062.133 7.121118068695068 LongInjection_Hminus_2023
we have a new trigger 2023-10-31 18:24:52 1698773092.133 0.0 Pbar_2023
we have a new trigger 2023-10-31 18:25:08 1698773107.733 7.909670352935791 NewOptics-Hminus_2023
we have a new trigger 2023-10-31 18:25:25 1698773124.533 6.342875957489014 LongInjection_Hminus_2023
we have a new trigger 2023-10-31 18:25:40 1698773140.133 8.83703327178955 NewOptics-Hminus_2023
we have a new trigger 2023-10-31 18:25:56 1698773155.733 10.031959533691406 LongInjection_Hminus_2023
we have

In [5]:
# pack daily data into text file     
# pickup signals       
BmExtrALPH  = 'LNE.APULB.0030:Acquisition:totalIntensitySingle' # Extr into 00 beam line (alpha,aegis,base ...
BmExtrGBAR  = 'LNE.APULB.5030:Acquisition:totalIntensitySingle' # Extr into 50 beam line (gbar, puma ...
SEM0008in   = 'LNE00.BSGW.0008:Acquisition:isMonitorIn' # SEM 0008 in
SEM0015in   = 'LNE00.BSGW.0015:Acquisition:isMonitorIn' # SEM 0015 in
SEM0025in   = 'LNE00.BSGW.0025:Acquisition:isMonitorIn' # SEM 0025 in
SEM5020in   = 'LNE50.BSGW.5020:Acquisition:isMonitorIn' # SEM 5020 in
SEM5060in   = 'LNE50.BSGW.5060:Acquisition:isMonitorIn' # SEM 5060 in
BmInjAD     = 'LNR.APULB.0227:Acquisition:totalIntensitySingle' # Inj into AD
CycleName   = 'ELENA.LSA:CYCLE'                         # Cycle name
 
print('Starting')

yr = 23
mt = 8
for dy in range(31,32):
    
    date = "20%02d-%02d-%02d " % (yr,mt,dy)    
    fila = "ELENA_BM_%02d_%02d_%02d.csv" % (yr,mt,dy)
    filb = "ELENA_Cycles_%02d_%02d_%02d.csv" % (yr,mt,dy)
    fold = "/eos/experiment/gbar/elena/data/%02d_%02d_%02d" % (yr,mt,dy)
    if not os.path.exists(fold):
       os.makedirs(fold)
    fdata = fold + "/" + fila
    fcycl = fold + "/" + filb
    
    t1 = date + '00:00:00.000'
    t2 = date + '23:59:59.999'
    
    dat  = ldb.get([BmExtrALPH,BmExtrGBAR,SEM0008in,SEM0015in,SEM0025in,SEM5020in,SEM5060in,BmInjAD],t1,t2)
    kys  = list(dat)
    
    cyc  = ldb.get([CycleName],t1,t2)
    kyc  = list(cyc)
    # print(kyc)
    # print(cyc[kyc[0]][0][-1])

    evs  = len(dat[kys[0]][0])
    evc  = len(cyc[kyc[0]][0])
    print(date,' : found ',evs,' and ',evc,' entries')
    
    # write a comma separated data file 
    fil = open(fdata, "w")
    # header
    fil.write("%d entries: Unix_Time,%s,%s,%s,%s,%s,%s,%s,%s\n" % (evs,BmExtrALPH,BmExtrGBAR,
                                       SEM0008in,SEM0015in,SEM0025in,SEM5020in,SEM5060in,BmInjAD))            
    for j in range(evs):
        # first is timestamp from extr to GBAR
        fil.write("%.3f,%.3f,%.3f," % (dat[kys[1]][0][j],dat[kys[0]][1][j],dat[kys[1]][1][j]))
        fil.write("%d,%d,%d,"       % (dat[kys[2]][1][j],dat[kys[3]][1][j],dat[kys[4]][1][j]))
        fil.write("%d,%d,%.3f\n"    % (dat[kys[5]][1][j],dat[kys[6]][1][j],dat[kys[7]][1][j]))        
    fil.close()    
    
    # write a comma separated file with time stamps and the Elena cycles 
    fil = open(fcycl, "w")
    # header
    fil.write("%d entries: Unix_Time,%s\n" % (evc,CycleName))            
    for j in range(evc):
        if cyc[kyc[0]][1][j] != 'ELENA_ZERO_CYCLE' :
            fil.write("%.3f,%s\n" % (cyc[kyc[0]][0][j],cyc[kyc[0]][1][j]))        
    fil.close()
    
print("finished")

Starting
2023-08-31   : found  1689  and  51306  entries
finished


In [ ]:
#d    = timedelta(minutes = 3)   # to make sure that we have at least 2 extractions
#ltrg = 0.0
#go   = True
#while go:                         #for i in range(10):  # 240
#    now  = datetime.now()  
#    dat  = ldb.get([BeamExtrALPH,BeamExtrGBAR,BeamInjection],now-d,now+d)
#    kys  = list(dat)
#    # dict_keys(['LNE.APULB.0030:Acquisition:totalIntensitySingle',
#    #            'LNE.APULB.5030:Acquisition:totalIntensitySingle',
#    #            'LNR.APULB.0227:Acquisition:totalIntensitySingle'])    
#    ntrg = dat[kys[1]][0][-1]                  # time of extr. to GBAR
#    cursor.execute("SELECT goElena FROM man")  # check if we want to stop
    go   = bool(cursor.fetchone()[0])          # bool(1) = True   int(bool(1)) = 1
    gbdb.commit()
    #print(go)
    if ntrg > ltrg and go:
        dt = datetime.fromtimestamp(round(ntrg,0))
        print("we have a new trigger",dt,dat[kys[1]][0][-1],dat[kys[1]][1][-1])  # second key, timestmp, last element I
        ltrg = ntrg
        cursor.execute("UPDATE elena SET extr03tme =" + str(dat[kys[0]][0][-1]))
        cursor.execute("UPDATE elena SET extr03sig =" + str(dat[kys[0]][1][-1]))  
        cursor.execute("UPDATE elena SET extr53tme =" + str(dat[kys[1]][0][-1]))
        cursor.execute("UPDATE elena SET extr53sig =" + str(dat[kys[1]][1][-1]))    
        cursor.execute("UPDATE elena SET inj227tme =" + str(dat[kys[2]][0][-1]))    
        cursor.execute("UPDATE elena SET inj227sig =" + str(dat[kys[2]][1][-1]))    
        gbdb.commit()       
    time.sleep(2)            
    
if gbdb is not None and gbdb.is_connected():
    gbdb.close()
    print('Connection mysql closed.')    
    


In [2]:
# Just some simple imports   
#import matplotlib.pyplot as plt
#old import datetime
#old import lmfit
plt.style.use('default')
plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams['lines.linewidth'] = 2

# search variables in the DB   #ldb.search("%totalIntensitySingle%")

In [3]:
#import pandas
import mysql.connector as msql
import datetime


# mysql -u gbar -pTo2023moko gbar -Bse "SELECT newdata FROM man;"

gbdb = msql.connect(host='pcnometh28.dyndns.cern.ch',database='gbar',user='gbar',password='To2023moko') 
if gbdb.is_connected():
    print('Connected to Gbar SC MySQL database')
    #cursor = gbsc.cursor(buffered=True)
    cursor = gbdb.cursor()
    #$cursor.execute("SELECT cpdate FROM man")
    #print(cursor.fetchall())
    upd_cpd = ("UPDATE elena SET extr53tme =" + str(1696864386.533))    
    cursor.execute(upd_cpd)
    gbdb.commit()
     
    #WHERE `run` = '345' AND `go` = '0' AND `stop` = '1' AND `nextTr` = '0' AND `newdata` = '1' AND `cpdate` = '1676042673' AND `Pb_on` = '12345678' AND `Hm_on` = '987654321' LIMIT 1;
  
if gbdb is not None and gbsc.is_connected():
    gbdb.close()
    print('Connection closed.')
    
# extr03tme extr03sig extr53tme extr53sig inj227tme inj227sig 
  
    #WHERE `run` = '345' AND `go` = '0' AND `stop` = '1' AND `nextTr` = '0' 
    #AND `newdata` = '1' AND `cpdate` = '1676042673' AND `Pb_on` = '12345678' AND `Hm_on` = '987654321' LIMIT 1;
     
    #print(dat['LNE.APULB.5030:Acquisition:totalIntensitySingle'][0][-1])

    #adat = dat.values()
    #print(adat)

    #for v in dat.values():
        #print(len(v))
        #print(v[0][-1],v[1][-1])
    
#dict_keys(['LNE.APULB.0030:Acquisition:totalIntensitySingle'])
#dict_items([('LNE.APULB.0030:Acquisition:totalIntensitySingle', (array([1.69684355e+09, 1.69684366e+09, 1.69684378e+09, 1.69684389e+09,
#       1.69684401e+09]), array([7.89372444, 8.47775269, 8.46801567, 8.59337997, 8.61419678])))])
#dict_values([(array([1.69684355e+09, 1.69684366e+09, 1.69684378e+09, 1.69684389e+09,
#       1.69684401e+09]), array([7.89372444, 8.47775269, 8.46801567, 8.59337997, 8.61419678]))])

#print("now =", now )  now = 2023-10-09 13:42:41.125025  
#dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
#print("date and time =", dt_string)  # date and time = 09/10/2023 13:42:41

    #print("selected key: ",kys[1])
    #print(type(dat))
    #print(dat.keys())
    #print(dat.items())
    #for k in dat:
    #    print(k)



#cursor = gbsc.cursor() cursor.execute("SELECT * FROM temp_C") data = np.asarray(cursor.fetchall()) evs = len(data)
#print("Total rows are: ", evs)
#print(data[evs-1,0])
#date = datetime.datetime.fromtimestamp(data[evs-1,0])
#print(date.strftime('Last entry: %Y-%m-%d %H:%M:%S'))
#cursor.close()

    
#import matplotlib.pyplot as plt    
#fig, ax = plt.subplots()
#ax.plot(data[:,0],data[:,1],'.')
#plt.show()

Connected to Gbar SC MySQL database
Connection closed.
1696864386.533


In [119]:

# old

#print(type(dat.values()))    
    
#print(dat['LNE.APULB.0030:Acquisition:totalIntensitySingle'])

#with open("simple.txt", "w") as fil:
#    fil.write("Hello world \n")
        
#fil = open("simple.txt", "w")
#for x in dat.items():
#  print(x)
#  fil.write(str(x))
#fil.close()    

#df = pd.DataFrame(dat.values())
#df.to_csv('test.txt',sep=',')
#df.to_csv('test.txt',sep='\t')

#BeamInjection = 'LNE.APULB.5030:INTENSITY'
#ADExtraction = 'DR.RF.NB-PART.100M_E:INTENSITY'

#SEMList_intensity = ['LNE50.BSGW.5020:FitAcq:gaussAmplitude','LNE50.BSGW.5060:FitAcq:gaussAmplitude']
#SEMList_position = ['LNE50.BSGW.5020:FitAcq:gaussMean','LNE50.BSGW.5060:FitAcq:gaussMean']
#SEMList_sigmas = ['LNE50.BSGW.5020:FitAcq:gaussSigma','LNE50.BSGW.5060:FitAcq:gaussSigma']

# GET the data from NXCALS via PyTimber
#SEM_intensity = ldb.get(SEMList_intensity, t1, t2)
#SEM_position  = ldb.get(SEMList_position, t1, t2)
#SEM_sigmas    = ldb.get(SEMList_sigmas, t1, t2)
#EXT_intensity = ldb.get(BeamInjection, t1,t2)
#AD_intensity = ldb.get(ADExtraction,t1,t2)

#InjInt = ldb.get(BeamInjection, t1,t2)
#AlpInt = ldb.get(BeamExtrALPH, t1,t2)
#GbrInt = ldb.get(BeamExtrGBAR, t1,t2)


Connected to Gbar SC MySQL database
[(1676042673,)]
Connection closed.
